In [15]:
import pandas as pd
from datetime import date
from bs4 import BeautifulSoup
import requests
from playwright.async_api import async_playwright
import time
import numpy as np
pd.set_option('display.max_colwidth', None)

In [16]:
# Take today's date and format it - must be used in the URLs.

date = date.today()

date = date.strftime("%d.%m.%Y")

print(date)

06.01.2023


In [17]:
# Read in the links and update with today's date
df = pd.read_csv("data/links.csv")

df = df.replace(to_replace=r'01.01.2023', value= date, regex=True)
df

,country,area,link
0,Albania,BZN AL,https://transparency.entsoe.eu/transmission-domain/r2/dayAheadPrices/show?name=&defaultValue=false&viewType=TABLE&areaType=BZN&atch=false&dateTime.dateTime=06.01.2023+00:00|CET|DAY&biddingZone.values=CTY|10YAL-KESH-----5!BZN|10YAL-KESH-----5&resolution.values=PT60M&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)
1,Austria,BZN AT,https://transparency.entsoe.eu/transmission-domain/r2/dayAheadPrices/show?name=&defaultValue=false&viewType=TABLE&areaType=BZN&atch=false&dateTime.dateTime=06.01.2023+00:00|CET|DAY&biddingZone.values=CTY|10YAT-APG------L!BZN|10YAT-APG------L&resolution.values=PT60M&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)
2,Austria,BZN DE-AT-LU,https://transparency.entsoe.eu/transmission-domain/r2/dayAheadPrices/show?name=&defaultValue=false&viewType=TABLE&areaType=BZN&atch=false&dateTime.dateTime=06.01.2023+00:00|CET|DAY&biddingZone.values=CTY|10YAT-APG------L!BZN|10Y1001A1001A63L&resolution.values=PT60M&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)
3,Belgium,BZN BE,https://transparency.entsoe.eu/transmission-domain/r2/dayAheadPrices/show?name=&defaultValue=false&viewType=TABLE&areaType=BZN&atch=false&dateTime.dateTime=06.01.2023+00:00|CET|DAY&biddingZone.values=CTY|10YBE----------2!BZN|10YBE----------2&resolution.values=PT60M&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)
4,Bosnia and Herzegovenia,BZN BA,https://transparency.entsoe.eu/transmission-domain/r2/dayAheadPrices/show?name=&defaultValue=false&viewType=TABLE&areaType=BZN&atch=false&dateTime.dateTime=06.01.2023+00:00|CET|DAY&biddingZone.values=CTY|10YBA-JPCC-----D!BZN|10YBA-JPCC-----D&resolution.values=PT60M&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)
...,...,...,...
74,Ukraine,BZN UA-DobTPP,https://transparency.entsoe.eu/transmission-domain/r2/dayAheadPrices/show?name=&defaultValue=false&viewType=TABLE&areaType=BZN&atch=false&dateTime.dateTime=06.01.2023+00:00|CET|DAY&biddingZone.values=CTY|10Y1001C--00003F!BZN|10Y1001A1001A869&resolution.values=PT60M&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)
75,United Kingdom,BZN GB,https://transparency.entsoe.eu/transmission-domain/r2/dayAheadPrices/show?name=&defaultValue=false&viewType=TABLE&areaType=BZN&atch=false&dateTime.dateTime=06.01.2023+00:00|CET|DAY&biddingZone.values=CTY|10Y1001A1001A92E!BZN|10YGB----------A&resolution.values=PT60M&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)
76,United Kingdom,BZN GB(ElecLink),https://transparency.entsoe.eu/transmission-domain/r2/dayAheadPrices/show?name=&defaultValue=false&viewType=TABLE&areaType=BZN&atch=false&dateTime.dateTime=06.01.2023+00:00|CET|DAY&biddingZone.values=CTY|10Y1001A1001A92E!BZN|11Y0-0000-0265-K&resolution.values=PT60M&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)
77,United Kingdom,BZN GB(IFA),https://transparency.entsoe.eu/transmission-domain/r2/dayAheadPrices/show?name=&defaultValue=false&viewType=TABLE&areaType=BZN&atch=false&dateTime.dateTime=06.01.2023+00:00|CET|DAY&biddingZone.values=CTY|10Y1001A1001A92E!BZN|10Y1001C--00098F&resolution.values=PT60M&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)


In [18]:
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless = True)
page = await browser.new_page()

url = df.iloc[0]['link']
await page.goto(url)

time.sleep(1)
await page.locator("#close-button").click()

In [19]:
# Create an empty dataframe for the scrape.
total_df = pd.DataFrame()
total_df

""


In [20]:
for index, row in df.iterrows():
    url = row['link']

    country = row['country']
    area = row['area']
    
    await page.goto(url)
    time.sleep(1)
    await page.get_by_text("PT60M").click()

    time.sleep(1)

    tables = pd.read_html(await page.content(), flavor = 'html5lib')
    data = tables[0]
    data.insert(0, 'country', country)
    data.insert(1, 'area', area)
    total_df = pd.concat([total_df, data], axis=0)
    
total_df

country          area            MTU  Day-ahead Price            
                                            MTU [Currency / MWh] [EUR / MWh]
0          Albania        BZN AL  00:00 - 01:00                -         NaN
1          Albania        BZN AL  01:00 - 02:00                -         NaN
2          Albania        BZN AL  02:00 - 03:00                -         NaN
3          Albania        BZN AL  03:00 - 04:00                -         NaN
4          Albania        BZN AL  04:00 - 05:00                -         NaN
..             ...           ...            ...              ...         ...
19  United Kingdom  BZN GB(IFA2)  19:00 - 20:00              n/e         NaN
20  United Kingdom  BZN GB(IFA2)  20:00 - 21:00              n/e         NaN
21  United Kingdom  BZN GB(IFA2)  21:00 - 22:00              n/e         NaN
22  United Kingdom  BZN GB(IFA2)  22:00 - 23:00              n/e         NaN
23  United Kingdom  BZN GB(IFA2)  23:00 - 00:00              n/e         NaN

[1896 rows x 5 columns]

In [21]:
total_df['date'] = pd.to_datetime('today').strftime("%m/%d/%Y")
total_df.columns = ['country', 'area', 'mtu', 'price(currency/mwh)', 'price(eur/mwh)', 'date']
total_df

# Standardize the missing values
total_df.replace({'-': np.nan}, inplace=True)
total_df.replace({'n/e': np.nan}, inplace=True)

# And reorder the columns
all_data = total_df[['date', 'country', 'area', 'mtu', 'price(currency/mwh)', 'price(eur/mwh)']]
all_data.head(50)

,date,country,area,mtu,price(currency/mwh),price(eur/mwh)
0,01/06/2023,Albania,BZN AL,00:00 - 01:00,NaN,NaN
1,01/06/2023,Albania,BZN AL,01:00 - 02:00,NaN,NaN
2,01/06/2023,Albania,BZN AL,02:00 - 03:00,NaN,NaN
3,01/06/2023,Albania,BZN AL,03:00 - 04:00,NaN,NaN
4,01/06/2023,Albania,BZN AL,04:00 - 05:00,NaN,NaN
5,01/06/2023,Albania,BZN AL,05:00 - 06:00,NaN,NaN
6,01/06/2023,Albania,BZN AL,06:00 - 07:00,NaN,NaN
7,01/06/2023,Albania,BZN AL,07:00 - 08:00,NaN,NaN
8,01/06/2023,Albania,BZN AL,08:00 - 09:00,NaN,NaN
9,01/06/2023,Albania,BZN AL,09:00 - 10:00,NaN,NaN


In [22]:
all_data.to_csv("data/europaenergy_today.csv", index=False)